# Normal

In [1]:
from models.lag_ml_model import scaling_function,preprocess_data,classifier_models,train_model,grid_optimize_model,random_optimize_model,get_param_grids,get_data

In [2]:
from main import normal_model,grid_model,random_model,backtest_loop,backtest,risk_backtest,risk_backtest_loop

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [4]:
df = get_data(ind=True)

In [5]:
df

,Date,Close,High,Low,Open,Volume,sentiment_score,shifted_direction,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,DMN_14,EMA_20,BBL_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,OBV,STOCHk_14_3_3,STOCHd_14_3_3
33,2015-02-20,211.240005,211.330002,208.729996,209.479996,140896400,0.887742,1.0,66.252193,2.187986,0.633537,1.554448,19.914895,28.000192,14.136314,206.836559,199.007933,212.568067,6.589370,0.902061,39682500.0,98.101258,98.208785
34,2015-02-23,211.210007,211.210007,210.479996,210.940002,74411100,0.887742,1.0,66.122789,2.267646,0.570558,1.697088,21.090534,27.229951,13.747447,207.253078,198.935802,213.264198,6.952158,0.856635,-34728600.0,98.153995,98.005801
35,2015-02-24,211.809998,212.050003,210.759995,211.119995,72472300,0.887742,1.0,67.490471,2.352079,0.523993,1.828086,22.580707,29.043759,13.089276,207.687070,198.844523,213.991477,7.338001,0.855979,37743700.0,98.384941,98.213398
36,2015-02-25,211.630005,212.240005,211.220001,211.660004,73061700,0.887742,1.0,66.621536,2.377067,0.439185,1.937882,24.011338,28.635755,12.576564,208.062588,199.162195,214.562805,7.444854,0.809566,-35318000.0,96.097360,97.545432
37,2015-02-26,211.380005,211.710007,210.649994,211.520004,72697900,0.887742,1.0,65.362771,2.349613,0.329384,2.020229,24.650475,27.433140,14.306716,208.378532,200.139276,214.709725,7.024459,0.771474,-108015900.0,93.013828,95.832043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,2025-02-21,599.940002,610.299988,599.469971,610.159973,76519800,0.887742,1.0,46.487773,2.775450,-0.207897,2.983347,9.926510,22.560590,30.389234,604.643138,596.578986,613.329009,2.768809,0.200657,481651000.0,75.901571,89.582316
2550,2025-02-24,597.210022,603.030029,596.489990,602.020020,50737200,0.887742,-1.0,44.002921,1.845712,-0.910108,2.755820,10.667655,20.880717,31.519243,603.935222,595.518941,613.313060,2.944019,0.095036,430913800.0,44.441729,71.908646
2551,2025-02-25,594.239990,597.890015,589.559998,597.150024,58266500,0.887742,-1.0,41.409640,0.859326,-1.517195,2.376521,12.150161,18.945692,36.307874,603.011867,594.437756,613.881244,3.218271,-0.010171,372647300.0,21.876638,47.406646
2552,2025-02-26,594.539978,599.580017,591.859985,595.929993,43321600,0.887742,-1.0,41.782844,0.100654,-1.820694,1.921348,13.194940,19.195055,33.234028,602.205020,593.077619,614.243377,3.506236,0.069091,415968900.0,15.037446,27.118604


In [6]:
X_train,X_test,y_train,y_test = preprocess_data(df,max_lag=10)

In [7]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1757, 31), (754, 31), (1757,), (754,))

In [8]:
X_train.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'sentiment_score', 'RSI_14',
       'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'ADX_14', 'DMP_14',
       'DMN_14', 'EMA_20', 'BBL_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0',
       'BBP_20_2.0', 'OBV', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'Close_lag1',
       'Close_lag2', 'Close_lag3', 'Close_lag4', 'Close_lag5', 'Close_lag6',
       'Close_lag7', 'Close_lag8', 'Close_lag9', 'Close_lag10'],
      dtype='object')

In [9]:
X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)

In [10]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1757, 31), (754, 31), (1757,), (754,))

In [11]:
results,y_pred_dict = normal_model(X_train_scaled,X_test_scaled,y_train,y_test)


Logistic Regression Performance:
Accuracy: 0.8873
Confusion Matrix:
[[207  60]
 [ 25 462]]
              precision    recall  f1-score   support

           0       0.89      0.78      0.83       267
           1       0.89      0.95      0.92       487

    accuracy                           0.89       754
   macro avg       0.89      0.86      0.87       754
weighted avg       0.89      0.89      0.89       754


Random Forest Performance:
Accuracy: 0.8753
Confusion Matrix:
[[233  34]
 [ 60 427]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       267
           1       0.93      0.88      0.90       487

    accuracy                           0.88       754
   macro avg       0.86      0.87      0.87       754
weighted avg       0.88      0.88      0.88       754


XGBoost Performance:
Accuracy: 0.9072
Confusion Matrix:
[[246  21]
 [ 49 438]]
              precision    recall  f1-score   support

           0       0.83      0.92

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [12]:
results

,Model,Accuracy,Confusion Matrix
0,Logistic Regression,0.887268,"[[207, 60], [25, 462]]"
1,Random Forest,0.875332,"[[233, 34], [60, 427]]"
2,XGBoost,0.907162,"[[246, 21], [49, 438]]"
3,SVM,0.891247,"[[212, 55], [27, 460]]"
4,Naive Bayes,0.893899,"[[232, 35], [45, 442]]"
5,Gradient Boosting,0.852785,"[[222, 45], [66, 421]]"
6,K-Nearest Neighbors,0.793103,"[[181, 86], [70, 417]]"
7,Decision Tree,0.822281,"[[221, 46], [88, 399]]"
8,AdaBoost,0.877984,"[[250, 17], [75, 412]]"
9,SGD Classifier,0.810345,"[[129, 138], [5, 482]]"


In [13]:
normal_df,bt_collection = backtest_loop(y_pred_dict,X_test)

Backtesting for Logistic Regression


Backtesting for Random Forest


Backtesting for XGBoost


Backtesting for SVM


Backtesting for Naive Bayes


...

Backtesting for Decision Tree


Backtesting for AdaBoost


Backtesting for SGD Classifier


Backtesting for MLP


Backtesting for LGBMClassifier


In [28]:
risk_normal_df,bt_collection = risk_backtest_loop(y_pred_dict,X_test,risk = 0.04 )
pd.merge(normal_df[['Return [%]','Model']],risk_normal_df[['Return [%]','Model']],on='Model')

Backtesting for Logistic Regression


Backtesting for Random Forest


Backtesting for XGBoost


Backtesting for SVM


Backtesting for Naive Bayes


...

Backtesting for AdaBoost


Backtesting for SGD Classifier


Backtesting for MLP


Backtesting for LGBMClassifier


,Return [%]_x,Model,Return [%]_y
0,22.012105,Logistic Regression,27.872101
1,19.991688,Random Forest,23.594886
2,47.595801,XGBoost,47.572001
3,22.016990,SVM,23.976625
4,20.744195,Naive Bayes,24.098495
5,29.871574,Gradient Boosting,28.608291
6,18.010182,K-Nearest Neighbors,17.224021
7,53.391604,Decision Tree,50.608787
8,26.195694,AdaBoost,25.882297
9,18.024504,SGD Classifier,18.844051


In [15]:
pd.merge(normal_df[['Return [%]','Model']],risk_normal_df[['Return [%]','Model']],on='Model')

,Return [%]_x,Model,Return [%]_y
0,22.012105,Logistic Regression,29.900396
1,19.991688,Random Forest,23.502968
2,47.595801,XGBoost,47.572001
3,22.016990,SVM,26.815200
4,20.744195,Naive Bayes,24.033875
5,29.871574,Gradient Boosting,28.592991
6,18.010182,K-Nearest Neighbors,17.848984
7,53.391604,Decision Tree,50.608787
8,26.195694,AdaBoost,25.866997
9,18.024504,SGD Classifier,19.661149


In [78]:
pd.merge(normal_df[['Return [%]','Model']],risk_normal_df[['Return [%]','Model']],on='Model')

,Return [%]_x,Model,Return [%]_y
0,22.012105,Logistic Regression,24.390110
1,29.836592,Random Forest,29.705491
2,47.595801,XGBoost,47.572001
3,22.016990,SVM,23.918493
4,20.744195,Naive Bayes,19.415795
5,34.122888,Gradient Boosting,33.938999
6,18.010182,K-Nearest Neighbors,17.312181
7,27.827468,Decision Tree,29.756067
8,26.195694,AdaBoost,25.866997
9,30.835509,SGD Classifier,32.937810


In [15]:
pd.merge(normal_df[['Return [%]','Model']],risk_normal_df[['Return [%]','Model']],on='Model')

,Return [%]_x,Model,Return [%]_y
0,17.098015,Logistic Regression,18.104215
1,26.822399,Random Forest,30.274599
2,47.257183,XGBoost,47.990588
3,18.888224,SVM,18.128817
4,22.287202,Naive Bayes,23.120800
5,35.211869,Gradient Boosting,34.816068
6,34.991374,K-Nearest Neighbors,35.004372
7,49.105667,Decision Tree,51.895077
8,27.898287,AdaBoost,27.907787
9,12.653558,SGD Classifier,11.761159


In [ ]:
from backtesting import Backtest, Strategy

def risk_backtest(y_pred, X_test, risk=0.20):
    def get_signal():
        return y_pred

    class MyStrategy(Strategy):
        # Risk settings:
        RISK_PER_TRADE = 1  # This will now be the percentage of equity to invest
        STOP_LOSS_PCT = risk  # 20% stop-loss from entry price

        def init(self):
            self.signal = self.I(get_signal)

        def next(self):
            price = self.data.Close[-1]
            current_equity = self.equity
            
            if self.signal[-1] == 1:
                if not self.position:
                    # Calculate shares based on risk percentage of total equity
                    investment_amount = current_equity * self.RISK_PER_TRADE
                    shares = int(investment_amount / price)
                    stop_loss = price * (1 - self.STOP_LOSS_PCT)

                    # Place a buy order with a protective stop-loss at 20% below entry
                    self.buy(size=shares, sl=stop_loss)
            else:
                # If signal == 0 and we have an open position, close it
                if self.position:
                    self.position.close()

    bt = Backtest(X_test, MyStrategy, cash=10000)
    stats = bt.run()
    return stats, bt

def risk_backtest_loop(y_pred_dict,X_test,risk = 0.20):
    stats_l=[]
    bt_collection = {}
    for name,y_pred in y_pred_dict.items():
        print(f"Backtesting for {name}")
        stats,bt = risk_backtest(y_pred,X_test,risk = risk)
        bt_collection[name] = bt
        stats['Model'] = name
        stats_l.append(stats)

    normal_df = pd.DataFrame(stats_l)

    return normal_df,bt_collection


In [51]:
from backtesting import Backtest, Strategy

def risk_backtest(y_pred, X_test, risk=0.20):
    def get_signal():
        return y_pred

    class MyStrategy(Strategy):
        # Risk settings:
        RISK_PER_TRADE = 1  # This will now be the percentage of equity to invest
        STOP_LOSS_PCT = risk  # 20% stop-loss from entry price

        def init(self):
            self.signal = self.I(get_signal)

        def next(self):
            price = self.data.Close[-1]
            current_equity = self.equity
            
            if self.signal[-1] == 1:
                if not self.position:
                    # Calculate shares based on risk percentage of total equity
                    investment_amount = current_equity * self.RISK_PER_TRADE
                    shares = int(investment_amount / price)
                    stop_loss = price * (1 - self.STOP_LOSS_PCT)

                    # Place a buy order with a protective stop-loss at 20% below entry
                    self.buy(size=shares, sl=stop_loss)
            else:
                # If signal == 0 and we have an open position, close it
                if self.position:
                    self.position.close()

    bt = Backtest(X_test, MyStrategy, cash=10000)
    stats = bt.run()
    return stats, bt

In [54]:
risk_backtest(y_pred_dict['Decision Tree'],X_test, risk=0.30)

(Start                     2022-03-18 00:00:00
 End                       2025-02-27 00:00:00
 Duration                   1077 days 00:00:00
 Exposure Time [%]                      64.682
 Equity Final [$]                  14873.71637
 Equity Peak [$]                   15299.37756
 Return [%]                           48.73716
 Buy & Hold Return [%]                31.61388
 Return (Ann.) [%]                    14.49731
 Volatility (Ann.) [%]                11.98309
 CAGR [%]                              9.73452
 Sharpe Ratio                          1.20981
 Sortino Ratio                         2.20297
 Calmar Ratio                          1.85417
 Alpha [%]                            36.72962
 Beta                                  0.37982
 Max. Drawdown [%]                    -7.81874
 Avg. Drawdown [%]                    -1.45344
 Max. Drawdown Duration      349 days 00:00:00
 Avg. Drawdown Duration       23 days 00:00:00
 # Trades                                   46
 Win Rate [%]

In [15]:
stats,_ = backtest(y_test,X_test)

In [16]:
stats = pd.DataFrame(stats).T
stats['Model'] = 'Actual'

In [17]:
stats

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Model
0,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,67.388363,15527.199982,15937.479156,55.272,31.613876,16.188467,12.23875,10.844105,1.322722,2.456555,2.698736,42.709006,0.397389,-5.998536,-1.441498,191 days 00:00:00,20 days 00:00:00,11,63.636364,20.579964,-3.072234,4.145516,162 days 00:00:00,65 days 00:00:00,8.961033,4.339341,2.193246,0.567772,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Actual


In [18]:
pd.concat([normal_df,stats],axis=0)

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Model
0,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,66.847091,11709.801483,12435.331635,17.098015,31.613876,5.529889,12.510375,3.762265,0.442024,0.674964,0.331741,1.344813,0.4983,-16.669293,-2.03446,443 days 00:00:00,34 days 00:00:00,20,40.0,10.876293,-11.878584,0.983801,126 days 00:00:00,35 days 00:00:00,1.871018,1.107855,0.951841,0.18269,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Logistic Regression
1,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,69.282815,12392.018524,12996.818268,23.920185,31.613876,7.587438,12.366643,5.146225,0.613541,0.979476,0.802296,9.107568,0.468548,-9.457161,-2.098744,246 days 00:00:00,30 days 00:00:00,27,62.962963,7.352972,-4.887157,0.892638,85 days 00:00:00,27 days 00:00:00,2.399381,0.930067,1.779614,0.372687,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Random Forest
2,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,70.230041,14725.718292,15445.717987,47.257183,31.613876,14.107531,12.841259,9.478059,1.09861,1.90202,1.553052,33.2715,0.442391,-9.083749,-1.647087,170 days 00:00:00,22 days 00:00:00,27,66.666667,9.629557,-4.098811,1.531281,87 days 00:00:00,27 days 00:00:00,4.743463,1.585123,2.548923,0.538695,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,XGBoost
3,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,69.959405,11888.822449,13086.261353,18.888224,31.613876,6.077297,12.721103,4.131286,0.477734,0.739101,0.389399,2.951967,0.504091,-15.606875,-2.072142,310 days 00:00:00,31 days 00:00:00,25,52.0,8.884654,-10.436167,0.786866,79 days 00:00:00,29 days 00:00:00,1.831801,0.866502,1.039503,0.229695,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,SVM
4,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,56.427605,12228.720154,12855.369141,22.287202,31.613876,7.101867,9.487132,4.820372,0.748579,1.168178,0.58793,13.623062,0.274061,-12.079439,-1.604886,490 days 00:00:00,34 days 00:00:00,32,56.25,7.610176,-4.785465,0.647305,59 days 00:00:00,18 days 00:00:00,2.220659,0.680245,1.456965,0.307107,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Naive Bayes
5,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,45.331529,13316.840027,13930.200134,33.1684,31.613876,10.260757,12.1267,6.932027,0.846129,1.417624,1.018591,19.665261,0.427127,-10.073485,-1.903949,421 days 00:00:00,26 days 00:00:00,22,59.090909,10.083252,-4.091032,1.086559,131 days 00:00:00,21 days 00:00:00,2.57249,1.140424,1.569365,0.36604,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Gradient Boosting
6,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,66.441137,13499.13739,14251.177429,34.991374,31.613876,10.773156,12.558759,7.272754,0.85782,1.420085,1.152085,20.876914,0.446464,-9.351009,-2.161563,141 days 00:00:00,30 days 00:00:00,27,59.259259,10.701142,-4.516816,1.143754,87 days 00:00:00,26 days 00:00:00,2.451399,1.205011,1.712131,0.341844,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,K-Nearest Neighbors
7,2022-03-18 00:00:00,2025-02-27 00:00:00,1077 days 00:00:00,50.473613,13264.147949,13556.967896,32.641479,31.613876,10.111791,10.936493,6.832876,0.924592,1.609816,0.944316,21.871391,0.340676,-10.708064,-1.881982,378 days 00:00:00,36 days 00:00:00,59,67.79661,8.055104,-6.104044,0.489017,52 days 00:00:00,8 days 00:00:00,2.320303,0.507686,2.017085,0.38177,MySt

In [19]:
y_pred_dict['XGBoost']

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [20]:
from backtesting import Backtest,Strategy

In [24]:
def get_signal():
    return y_pred_dict['XGBoost']

get_signal()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [32]:
y_pred_dict.keys()

dict_keys(['Logistic Regression', 'Random Forest', 'XGBoost', 'SVM', 'Naive Bayes', 'Gradient Boosting', 'K-Nearest Neighbors', 'Decision Tree', 'AdaBoost', 'SGD Classifier', 'MLP', 'LGBMClassifier'])

In [33]:
def get_signal():
    return y_pred_dict['SVM']

class MyStrategy(Strategy):
    def init(self):
        self.signal = self.I(get_signal)

    def next(self):
        if self.signal[-1] == 1:
            if not self.position:
                self.buy()
        elif self.signal[-1] == 0:
            if self.position:
                self.position.close()

bt = Backtest(X_test,MyStrategy,cash=10000)
bt.run()        

Start                     2022-03-18 00:00:00
End                       2025-02-27 00:00:00
Duration                   1077 days 00:00:00
Exposure Time [%]                     69.9594
Equity Final [$]                  11888.82245
Equity Peak [$]                   13086.26135
Return [%]                           18.88822
Buy & Hold Return [%]                31.61388
Return (Ann.) [%]                      6.0773
Volatility (Ann.) [%]                 12.7211
CAGR [%]                              4.13129
Sharpe Ratio                          0.47773
Sortino Ratio                          0.7391
Calmar Ratio                           0.3894
Alpha [%]                             2.95197
Beta                                  0.50409
Max. Drawdown [%]                   -15.60688
Avg. Drawdown [%]                    -2.07214
Max. Drawdown Duration      310 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                   25
Win Rate [%]                      

In [34]:
def get_signal():
    return y_pred_dict['SVM']

class MyStrategy(Strategy):
    # Risk settings:
    RISK_PER_TRADE = 0.02   # 2% risk per trade
    STOP_LOSS_PCT  = 0.05   # 5% stop-loss from entry price

    def init(self):
        self.signal = self.I(get_signal)

    def next(self):
        price = self.data.Close[-1]
        current_equity = self.equity
        # Determine how many shares we can buy so that, if stopped out 5% below entry,
        # we lose no more than 2% of current equity.
        # Risk in dollars = current_equity * RISK_PER_TRADE
        # Stop-loss distance = price * STOP_LOSS_PCT
        # => position_size = (Risk in dollars) / (Stop-loss distance)
        if self.signal[-1] == 1:
            if not self.position:
                risk_dollars = current_equity * self.RISK_PER_TRADE
                stop_dist    = price * self.STOP_LOSS_PCT
                shares       = risk_dollars / stop_dist if stop_dist > 0 else 0

                # Optional rounding to get whole shares:
                shares = int(shares)

                # Place a buy order with a protective stop-loss
                self.buy(size=shares, sl=price * (1 - self.STOP_LOSS_PCT))
        else:
            # If signal == 0 and we have an open position, close it
            if self.position:
                self.position.close()

bt = Backtest(X_test, MyStrategy, cash=10000)
stats = bt.run()
print(stats)

Start                     2022-03-18 00:00:00
End                       2025-02-27 00:00:00
Duration                   1077 days 00:00:00
Exposure Time [%]                    69.82409
Equity Final [$]                   10909.8553
Equity Peak [$]                   11319.84493
Return [%]                            9.09855
Buy & Hold Return [%]                31.61388
Return (Ann.) [%]                     3.01402
Volatility (Ann.) [%]                 4.77037
CAGR [%]                              2.05846
Sharpe Ratio                          0.63182
Sortino Ratio                          0.9694
Calmar Ratio                          0.62236
Alpha [%]                             2.99634
Beta                                  0.19302
Max. Drawdown [%]                    -4.84287
Avg. Drawdown [%]                    -0.85318
Max. Drawdown Duration      136 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   26
Win Rate [%]                      

In [ ]:


def get_signal():
    return y_pred_dict['SVM']

class MyStrategy(Strategy):
    # Modified risk settings:
    RISK_PER_TRADE = 0.02   # Keeping 2% risk per trade
    STOP_LOSS_PCT  = 0.20   # Changed to 20% stop-loss from entry price

    def init(self):
        self.signal = self.I(get_signal)

    def next(self):
        price = self.data.Close[-1]
        current_equity = self.equity
        
        if self.signal[-1] == 1:
            if not self.position:
                risk_dollars = current_equity * self.RISK_PER_TRADE
                stop_dist    = price * self.STOP_LOSS_PCT
                shares       = risk_dollars / stop_dist if stop_dist > 0 else 0

                # Optional rounding to get whole shares:
                shares = int(shares)

                # Place a buy order with a protective stop-loss
                self.buy(size=shares, sl=price * (1 - self.STOP_LOSS_PCT))
        else:
            # If signal == 0 and we have an open position, close it
            if self.position:
                self.position.close()

bt = Backtest(X_test, MyStrategy, cash=10000)
stats = bt.run()
print(stats)

Start                     2022-03-18 00:00:00
End                       2025-02-27 00:00:00
Duration                   1077 days 00:00:00
Exposure Time [%]                     69.9594
Equity Final [$]                  10154.07019
Equity Peak [$]                   10212.64014
Return [%]                             1.5407
Buy & Hold Return [%]                31.61388
Return (Ann.) [%]                     0.52274
Volatility (Ann.) [%]                 0.97308
CAGR [%]                              0.35839
Sharpe Ratio                           0.5372
Sortino Ratio                          0.7878
Calmar Ratio                          0.37074
Alpha [%]                              0.2608
Beta                                  0.04049
Max. Drawdown [%]                    -1.40997
Avg. Drawdown [%]                    -0.16665
Max. Drawdown Duration      443 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                   25
Win Rate [%]                      

# Random

In [19]:
rand_results,y_pred_dict_rand = random_model(X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)

Working on optimizing with RandomSearchCV
Optimizing Logistic Regression...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'C': 0.1}
Optimizing Random Forest...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Random Forest: {'n_estimators': 100, 'min_samples_split': 2, 'max_features': 'log2', 'max_depth': None}
Optimizing XGBoost...
No hyperparameters to tune for XGBoost
Optimizing SVM...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for SVM: {'kernel': 'linear', 'gamma': 'scale', 'C': 0.1}
Optimizing Naive Bayes...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters for Naive Bayes: {'alpha': 0.1}
Optimizing Gradient Boosting...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for Gradient Boosting: {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': 3, 'learning_rate': 0.05}
Optimizing K-Nearest Neighbors...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for K-Nearest Neighbors: {'weights': 'distance', 'p': 2, 'n_neighbors': 7, 'algorithm': 'kd_tree'}
Optimizing Decision Tree...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Decision Tree: {'min_samples_split': 10, 'max_features': None, 'max_depth': 5, 'criterion': 'entropy'}
Optimizing AdaBoost...
Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for AdaBoost: {'n_estimators': 50, 'learning_rate': 0.1}
Optimizing SGD Classifier...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\linear_model\_stochast

Best parameters for SGD Classifier: {'penalty': 'l2', 'loss': 'log_loss', 'learning_rate': 'optimal', 'alpha': 0.001}
Optimizing MLP...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for MLP: {'solver': 'sgd', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (100,), 'alpha': 0.001, 'activation': 'tanh'}
Optimizing LGBMClassifier...
[LightGBM] [Info] Number of positive: 1095, number of negative: 627
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001616 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20655
[LightGBM] [Info] Number of data points in the train set: 1722, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.635889 -> initscore=0.557563
[LightGBM] [Info] Start training from score 0.557563
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [23]:
rand_results

,Model,Accuracy,Confusion Matrix
0,Logistic Regression,0.875507,"[[290, 55], [37, 357]]"
1,Random Forest,0.826793,"[[227, 118], [10, 384]]"
2,XGBoost,0.887686,"[[289, 56], [27, 367]]"
3,SVM,0.876861,"[[284, 61], [30, 364]]"
4,Naive Bayes,0.849797,"[[247, 98], [13, 381]]"
5,Gradient Boosting,0.884980,"[[292, 53], [32, 362]]"
6,K-Nearest Neighbors,0.857916,"[[264, 81], [24, 370]]"
7,Decision Tree,0.868742,"[[292, 53], [44, 350]]"
8,AdaBoost,0.882273,"[[298, 47], [40, 354]]"
9,SGD Classifier,0.893099,"[[299, 46], [33, 361]]"


In [ ]:
random_optimize_model()

In [24]:
random_df,bt_collection = backtest_loop(y_pred_dict_rand,X_test)

Backtesting for Logistic Regression


Backtesting for Random Forest


Backtesting for XGBoost


Backtesting for SVM


Backtesting for Naive Bayes


...

Backtesting for Decision Tree


Backtesting for AdaBoost


Backtesting for SGD Classifier


Backtesting for MLP


Backtesting for LGBMClassifier


In [29]:
pd.concat([random_df,normal_df,stats],axis=0,keys=['Random','Normal','Actual']).sort_values(by='Return [%]',ascending=False)

Start                  End            Duration  \
Actual 0   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Random 1   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       5   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Normal 9   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       0   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       11  2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Random 11  2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       4   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Normal 4   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       6   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Random 10  2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       7   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Normal 10  2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Random 6   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Normal 5   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       3   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       1   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       7   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       2   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Random 2   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Normal 8   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
Random 9   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       3   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       0   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   
       8   2022-03-18 00:00:00  2025-02-27 00:00:00  1077 days 00:00:00   

          Exposure Time [%] Equity Final [$] Equity Peak [$] Return [%]  \
Actual 0          51.962111     17565.242096    18567.572235  75.652421   
Random 1          68.606225     14386.921524    15110.780411  43.869215   
       5          57.780785     13325.667465    14151.458801  33.256675   
Normal 9          64.140731     13318.750397    14616.060608  33.187504   
       0          62.516915     13207.580185    14186.071014  32.075802   
       11         59.269283     13056.729919    14570.940155  30.567299   
Random 11         59.269283     13056.729919    14570.940155  30.567299   
       4          65.899865     12858.899521     15330.73909  28.588995   
Normal 4          65.899865     12858.899521     15330.73909  28.588995   
       6          61.705007     12629.009506    13708.208969  26.290095   
Random 10         65.493911     12310.279388    14322.878189  23.102794   
       7          56.833559     12067.851608    13915.790359  20.678516   
Normal 10         54.262517     11944.571487    14206.991501  19.445715   
Random 6          62.787551     11891.309418    14608.229004  18.913094   
Normal 5          60.487145     11865.396271    14646.997223  18.653963   
       3          60.216509     11828.309921     13795.96019  18.283099   
       1          64.682003     11692.869415    12558.320175  16.928694   
       7          53.856563     11628.970322    13205.791046  16.289703   
       2          59.539919     11544.361633    13230.371307  15.443616   
Random 2          59.539919     11544.361633    13230.371307  15.443616   
Normal 8          56.156969     11515.740311     13894.90979  15.157403   
Random 9          56.292287     11344.151047    12756.790726   13.44151   
       3          58.728011     10734.139923    12823.980484   7.341399   
       0          57.104195     10727.600891    12394.001099   7.276009   
       8          56.156969     10568.612335    12974.912125   5.686123   

          Buy & Hold Return [%] Return (Ann.) [%] Volatility (Ann.) [%]  \
Actual 0              44.712775         21.17900

# Random